In [1]:
import time
import random
import math

In [2]:
people = [('Seymour','BOS'),
          ('Franny','DAL'),
          ('Zooey','CAK'),
          ('Walt','MIA'),
          ('Buddy','ORD'),
          ('Les','OMA')]
# Laguardia
destination='LGA'

In [4]:
flights={}
# 
for line in open('./chapter5/schedule.txt'):
    origin,dest,depart,arrive,price=line.strip().split(',')
    flights.setdefault((origin,dest),[])

    # Add details to the list of possible flights
    flights[(origin,dest)].append((depart,arrive,int(price)))

def getminutes(t):
    x=time.strptime(t,'%H:%M')
    return x[3]*60+x[4]

In [13]:
def printschedule(r):
    for d in range(int(len(r)/2)):
        name = people[d][0]
        origin = people[d][1]
        out = flights[(origin, destination)][r[2 * d]]
        ret = flights[(destination, origin)][r[2 * d + 1]]
        print(('%10s%10s %5s-%5s $%3s %5s-%5s $%3s') % (name, origin,
                                                      out[0], out[1], out[2], 
                                                      ret[0], ret[1], ret[2]))

In [14]:
s = [1, 4, 3, 2, 7, 3, 6, 3, 2, 4, 5, 3]
printschedule(s)

   Seymour       BOS  8:04-10:11 $ 95 12:08-14:05 $142
    Franny       DAL 10:30-14:57 $290  9:49-13:51 $229
     Zooey       CAK 17:08-19:08 $262 10:32-13:16 $139
      Walt       MIA 15:34-18:11 $326 11:08-14:38 $262
     Buddy       ORD  9:42-11:32 $169 12:08-14:47 $231
       Les       OMA 13:37-15:08 $250 11:07-13:24 $171


In [25]:
def schedulecost(sol):
    totalprice = 0
    latestarrival = 0
    earliestdep = 24 * 60
    
    for d in range(int(len(sol) / 2)):
        origin = people[d][1]
        outbound = flights[(origin, destination)][int(sol[2 * d])]
        returnf  = flights[(destination, origin)][int(sol[2 * d + 1])]
        
        totalprice += outbound[2]
        totalprice += returnf[2]
        
        if latestarrival < getminutes(outbound[1]): latestarrival = getminutes(outbound[1])
        if earliestdep > getminutes(returnf[0]): earliestdep = getminutes(returnf[0])
            
    totalwait = 0
    for d in range(int(len(sol) / 2)):
        origin = people[d][1]
        outbound = flights[(origin, destination)][int(sol[2 * d])]
        returnf = flights[(destination, origin)][int(sol[2 * d + 1])]
        totalwait += latestarrival - getminutes(outbound[1])
        totalwait += getminutes(returnf[0]) - earliestdep
    
    if latestarrival > earliestdep: totalprice += 50
        
    return totalprice + totalwait

In [26]:
schedulecost(s)

4635

In [31]:
def randomoptimize(domain, costf):
    best = 999999999
    bestr = None
    for i in range(1000):
        r = [random.randint(domain[i][0], domain[i][1])
             for i in range(len(domain))]
        
        cost = costf(r)
        
        if cost < best:
            best = cost
            bestr = r
    return r

In [33]:
domain = [(0, 9)] * (len(people) * 2)
s = randomoptimize(domain, schedulecost)
schedulecost(s)

5818

In [34]:
printschedule(s)

   Seymour       BOS 15:27-17:18 $151 13:39-15:30 $ 74
    Franny       DAL 10:30-14:57 $290  9:49-13:51 $229
     Zooey       CAK 18:35-20:28 $204 10:32-13:16 $139
      Walt       MIA 12:05-15:30 $330  9:25-12:46 $295
     Buddy       ORD 11:01-12:39 $260 18:33-20:22 $143
       Les       OMA  6:11- 8:31 $249 14:05-15:47 $226


In [46]:
def hillclimb(domain,costf):
    # Create a random solution
    sol=[random.randint(domain[i][0],domain[i][1]) for i in range(len(domain))]
    
    # Main loop
    while 1:
        # Create list of neighboring solutions
        neighbors=[]
    
        for j in range(len(domain)):
            # One away in each direction
            if sol[j]>domain[j][0]:
                neighbors.append(sol[0:j]+[sol[j]-1]+sol[j+1:]) # 
            if sol[j]<domain[j][1]:
                neighbors.append(sol[0:j]+[sol[j]+1]+sol[j+1:])

        # See what the best solution amongst the neighbors is
        current=costf(sol)
        best=current
        for j in range(len(neighbors)):
            cost=costf(neighbors[j])
            if cost<best:
                best=cost
                sol=neighbors[j]

        # If there's no improvement, then we've reached the top
        if best==current: break

    return sol

In [52]:
s = hillclimb(domain, schedulecost)
schedulecost(s)

4385

In [51]:
printschedule(s)

   Seymour       BOS 13:40-15:37 $138 10:33-12:03 $ 74
    Franny       DAL 10:30-14:57 $290 10:51-14:16 $256
     Zooey       CAK 13:40-15:38 $137 13:37-15:33 $142
      Walt       MIA 11:28-14:40 $248 12:37-15:05 $170
     Buddy       ORD 14:22-16:32 $126 10:33-13:11 $132
       Les       OMA 15:03-16:42 $135 15:07-17:21 $129


In [55]:
# simulated annealing

def annealingoptimize(domain, costf, T=10000.0, cool=0.95, step=1):
    # get random opt value
    vec = [float(random.randint(domain[i][0], domain[i][1]))
          for i in range(len(domain))]
    
    while T > 0.1:
        # get index
        i = random.randint(0, len(domain) -1)
        # selection direction update index
        direction = random.randint(-step, step)
        
        # built solution list
        vecb = vec[:]
        vecb[i] += direction
        if vecb[i] < domain[i][0]: vecb[i] = domain[i][0]
        elif vecb[i] > domain[i][1]: vecb[i] = domain[i][1]
            
        # calculate current cost
        ea = costf(vec)
        eb = costf(vecb)
        
        #
        if (eb < ea or random.random() < pow(math.e, -(eb-ea)/T)):
            vec = vecb
        
        # decrease temp
        t = T * cool
    return vec

In [56]:
s = annealingoptimize(domain, schedulecost)
schedulecost(s)

KeyboardInterrupt: 

In [ ]:
printschedule(s)